In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# corerenza di utilizzo nomi per etichette

#### label_id    ( etichetta codice, es:  "A03"  )
###  label_text  ( etichetta testuale, es:  "Colera"  )
###  source_text ( testo finale da embeddare: "Colera, malattia colerica ..."" )

## Labels

### Labels data cleaning

In [21]:
import pandas as pd
import matplotlib.pyplot as plt



malat_Cat = pd.read_excel("../data_icd10/Codici-ICD10-categorie_mlt_tmt.xlsx", sheet_name='categorie_mlt_tmt',header=1)
malat_SubCat = pd.read_excel("../data_icd10/Codici-ICD10-categorie_mlt_tmt.xlsx", sheet_name='sottocategorie_mlt_tmt',header=1)

##############################
# Rename columns, but keep old names:
column_name_reference_CAT = {
    "Codice Categoria Malattie e Traumatismi": "Codice_categoria",
    "Capitolo": "Capitolo",
    "Blocchi di categorie": "Blocco",
    "Descrizione delle categorie di Malattie e traumatismi": "Descr_Categ"
}
# Apply renaming
malat_Cat = malat_Cat.rename(columns=column_name_reference_CAT)

column_name_reference_SubCat = {
    "Codice Categoria Malattie e traumatismi": "Codice_categoria",
    "Codice sottocategoria Malattie e Traumatismi": "Codice_SottoCategoria",
    "Descrizione delle sottocategorie di Malattie e traumatismi": "Descr_SottoCateg"
}

#
malat_SubCat = malat_SubCat.rename(columns=column_name_reference_SubCat)

malat_SubCat['Codice_SottoCategoria']= malat_SubCat['Codice_SottoCategoria'].astype('Int64')
malat_Cat['Capitolo'] = malat_Cat['Capitolo'].astype('Int64')
malat_Cat['Blocco'] = malat_Cat['Blocco'].astype('Int64')

# elimina righe vuote
malat_Cat = malat_Cat.dropna()
malat_SubCat = malat_SubCat.dropna()

display(malat_Cat.head(3))
print('-------------')
display(malat_SubCat.head(7))




,Codice_categoria,Capitolo,Blocco,Descr_Categ
0,A00,1,1,Colera
1,A01,1,1,Febbre tifoide e paratifo
2,A02,1,1,Altre infezioni da salmonella


-------------


,Codice_categoria,Codice_SottoCategoria,Descr_SottoCateg
0,A00,0,"Colera da Vibrio cholerae 01, biotipo del colera"
1,A00,1,"Colera da Vibrio cholerae 01, biotipo eltor"
2,A00,9,Colera non specificato
3,A01,0,Febbre tifoide
4,A01,1,Paratifo A
5,A01,2,Paratifo B
6,A01,3,Paratifo C


In [22]:
# Goal is to concatenate all subcat text for each categ.
# Concatenate text in each category with groupby and join

df_subcat_text= malat_SubCat.groupby('Codice_categoria')['Descr_SottoCateg'].apply(lambda text: ' '.join(text))

# and also remake it a proper dataframe with reset index, 
# so that it can be merged back
df_subcat_text= df_subcat_text.reset_index()
df_subcat_text.columns = ['Codice_categoria', 'Descr_SottoCateg_Estesa']

display(malat_SubCat.head(5))
df_subcat_text.head(3)

,Codice_categoria,Codice_SottoCategoria,Descr_SottoCateg
0,A00,0,"Colera da Vibrio cholerae 01, biotipo del colera"
1,A00,1,"Colera da Vibrio cholerae 01, biotipo eltor"
2,A00,9,Colera non specificato
3,A01,0,Febbre tifoide
4,A01,1,Paratifo A


,Codice_categoria,Descr_SottoCateg_Estesa
0,A00,"Colera da Vibrio cholerae 01, biotipo del cole..."
1,A01,Febbre tifoide Paratifo A Paratifo B Paratifo ...
2,A02,Enterite da salmonella Setticemia da salmonell...


In [23]:
#### merge

malat_Cat_full = malat_Cat.merge(
    how='left',
    right= df_subcat_text,
    on= 'Codice_categoria',
)

# sostituisce NaN con stringa vuota per fare unione finale stringhe
malat_Cat_full['Descr_SottoCateg_Estesa'] =  malat_Cat_full['Descr_SottoCateg_Estesa'].fillna('')
malat_Cat_full.tail()

# crea descrizione totale unendo anche descrizione base della categoria 
malat_Cat_full['Descr_totale_Estesa'] =  malat_Cat_full['Descr_Categ'] + ': ' + malat_Cat_full['Descr_SottoCateg_Estesa']
malat_Cat_full.tail(3)

,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
1949,Y96,20,35,Condizione associata al lavoro,Condizione associata al lavoro,Condizione associata al lavoro: Condizione ass...
1950,Y97,20,35,Condizione associata all'inquinamento ambientale,Condizione associata all'inquinamento ambientale,Condizione associata all'inquinamento ambienta...
1951,Y98,20,35,Condizione associata allo stile di vita,Condizione associata allo stile di vita,Condizione associata allo stile di vita: Condi...


#### Final Cleaned Dataframe to use for collection

In [24]:
malat_Cat_full.head()

,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
0,A00,1,1,Colera,"Colera da Vibrio cholerae 01, biotipo del cole...","Colera: Colera da Vibrio cholerae 01, biotipo ..."
1,A01,1,1,Febbre tifoide e paratifo,Febbre tifoide Paratifo A Paratifo B Paratifo ...,Febbre tifoide e paratifo: Febbre tifoide Para...
2,A02,1,1,Altre infezioni da salmonella,Enterite da salmonella Setticemia da salmonell...,Altre infezioni da salmonella: Enterite da sal...
3,A03,1,1,Shigellosi,Shigellosi da Shigella dysenteriae Shigellosi ...,Shigellosi: Shigellosi da Shigella dysenteriae...
4,A04,1,1,Altre infezioni batteriche intestinali,Infezione da Escherichia coli enteropatogena I...,Altre infezioni batteriche intestinali: Infezi...


### Labels create collection

In [25]:
# Dai puliti:
malat_Cat_full.head(3)

,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
0,A00,1,1,Colera,"Colera da Vibrio cholerae 01, biotipo del cole...","Colera: Colera da Vibrio cholerae 01, biotipo ..."
1,A01,1,1,Febbre tifoide e paratifo,Febbre tifoide Paratifo A Paratifo B Paratifo ...,Febbre tifoide e paratifo: Febbre tifoide Para...
2,A02,1,1,Altre infezioni da salmonella,Enterite da salmonella Setticemia da salmonell...,Altre infezioni da salmonella: Enterite da sal...


In [6]:
# per usare i moduli 
# da temrinal fare pip install -e . 
# ( "e"  sta per editing )
# cosi installa tutto e perfmettte di fare modifiche ai moduli e riavviare e riusarli



In [7]:
df_copy = malat_Cat_full.copy()
df_copy.head(2)


,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
0,A00,1,1,Colera,"Colera da Vibrio cholerae 01, biotipo del cole...","Colera: Colera da Vibrio cholerae 01, biotipo ..."
1,A01,1,1,Febbre tifoide e paratifo,Febbre tifoide Paratifo A Paratifo B Paratifo ...,Febbre tifoide e paratifo: Febbre tifoide Para...


In [8]:
from semantic_search.data import Document, Corpus

corpus_df= Corpus.from_dataframe(
    df=df_copy,
    source_text_col='Descr_totale_Estesa',
    label_text_col='Descr_Categ',
    label_id_col='Codice_categoria',
    father_label_id_col='Blocco',
    g_father_label_id_col='Capitolo'
      )

corpus_df.documents[:5]

[Document(id=0, content='Colera: Colera da Vibrio cholerae 01, biotipo del colera Colera da Vibrio cholerae 01, biotipo eltor Colera non specificato', metadata={'label_id': 'A00', 'label_text': 'Colera', 'father_label_id': 1, 'g_father_label_id': 1}),
 Document(id=1, content='Febbre tifoide e paratifo: Febbre tifoide Paratifo A Paratifo B Paratifo C Paratifo non specificato', metadata={'label_id': 'A01', 'label_text': 'Febbre tifoide e paratifo', 'father_label_id': 1, 'g_father_label_id': 1}),
 Document(id=2, content='Altre infezioni da salmonella: Enterite da salmonella Setticemia da salmonella Infezioni localizzate da salmonella Altre infezioni da salmonella, specificate Infezione da salmonella, non specificata', metadata={'label_id': 'A02', 'label_text': 'Altre infezioni da salmonella', 'father_label_id': 1, 'g_father_label_id': 1}),
 Document(id=3, content='Shigellosi: Shigellosi da Shigella dysenteriae Shigellosi da Shigella flexneri Shigellosi da Shigella boydii Shigellosi da Shi

In [3]:
from semantic_search.qdrant import EmbeddingModel, CollectionMananger

cm = CollectionMananger()

cm.list_collections()

['istat-data-384',
 'test-new-script',
 'test-collection-dev',
 'istat-data-v2-3072',
 'istat-data-open',
 'test-collection-01',
 'istat-icd10_bis',
 'istat-data-3072',
 'istat-data',
 'istat-icd10',
 'istat-ateco',
 'dummy_object_labels',
 'istat-data-AP',
 'istat-icd10_Factory_0']

In [10]:
# cm.create(
#     name="istat-icd10_Factory_0",
#     corpus=corpus_df,
#     model_id="text-embedding-ada-002",
#     model_type="openai",
#     vector_size=1536,     
#     overwrite=True
# )

Uploading batches to istat-icd10_Factory_0: 100%|██████████| 61/61 [00:57<00:00,  1.07it/s]


In [ ]:
# cm.create(
#     name="istat-icd10_Factory_0",
#     corpus=corpus_df,
#     model_id="text-embedding-ada-002",
#     model_type="openai",
#     vector_size=1536,     
#     overwrite=True,
#     embed_batch_size= 128
# )

Uploading batches to istat-icd10_Factory_0: 100%|██████████| 61/61 [02:15<00:00,  2.23s/it]


In [4]:
cm.get_collection_info('istat-icd10_Factory_0')

{'name': 'istat-icd10_Factory_0',
 'status': 'green',
 'vector_size': 1536,
 'distance': 'Cosine',
 'points_count': 1952}

#### Test Seeking on collection

In [6]:
# Single Query seeking 

from semantic_search.qdrant import SemanticSeeker

seeker = SemanticSeeker(
    collection_name="istat-icd10_Factory_0",
    model_id="text-embedding-ada-002",
    model_type="openai",
    vector_size=1536,       # ada-002 will default to its fixed 1536
)

# 2) Run the search
top_findings = seeker.search(query='Scarlattina', top_k=3 )
top_findings.results



Creating embeddings...: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]


[RetrievedPoint(id=32, metadata={'label_id': 'A38', 'label_text': 'Scarlattina', 'father_label_id': 4, 'g_father_label_id': 1, 'source_text': 'Scarlattina: Scarlattina'}, score=0.9711024),
 RetrievedPoint(id=159, metadata={'label_id': 'B86', 'label_text': 'Scabbia', 'father_label_id': 18, 'g_father_label_id': 1, 'source_text': 'Scabbia: Scabbia'}, score=0.8494627),
 RetrievedPoint(id=898, metadata={'label_id': 'L84', 'label_text': "Calli e callosita'", 'father_label_id': 8, 'g_father_label_id': 12, 'source_text': "Calli e callosita': Calli e callosita'"}, score=0.8409397)]

In [7]:
# Multiple Query seeking 
from semantic_search.qdrant import SemanticSeeker

seeker = SemanticSeeker(
    collection_name="istat-icd10_Factory_0",
    model_id="text-embedding-ada-002",
    model_type="openai",
    vector_size=1536,       # ada-002 will default to its fixed 1536
)

queries_list  = ["Scarlattina", "Epilessia", "Polmonite"]
results= seeker.search_many(queries= queries_list, top_k= 3)
results


Creating embeddings...: 100%|██████████| 1/1 [00:00<00:00,  1.83it/s]


[SearchOutput(results=[RetrievedPoint(id=32, metadata={'label_id': 'A38', 'label_text': 'Scarlattina', 'father_label_id': 4, 'g_father_label_id': 1, 'source_text': 'Scarlattina: Scarlattina'}, score=0.97108066), RetrievedPoint(id=159, metadata={'label_id': 'B86', 'label_text': 'Scabbia', 'father_label_id': 18, 'g_father_label_id': 1, 'source_text': 'Scabbia: Scabbia'}, score=0.84946114), RetrievedPoint(id=898, metadata={'label_id': 'L84', 'label_text': "Calli e callosita'", 'father_label_id': 8, 'g_father_label_id': 12, 'source_text': "Calli e callosita': Calli e callosita'"}, score=0.8409029)]),
 SearchOutput(results=[RetrievedPoint(id=519, metadata={'label_id': 'G40', 'label_text': 'Epilessia', 'father_label_id': 6, 'g_father_label_id': 6, 'source_text': 'Epilessia: Epilessia e sindromi epilettiche idiopatiche, definite per localizzazione (focale) (parziale), con crisi ad esordio focale Epilessia e sindromi epilettiche sintomatiche, definite per localizzazione (focale(i)) (parziale(i

## Test-Data

### Test-Data Cleaning

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Dati Reali da testare:
df_note= pd.read_excel('../data_icd10/TRAINING_medcod_012022.xlsx')
df_note.head(4)
print(df_note.shape)

(253626, 7)


In [3]:
df_note['linecoded']= df_note['linecoded'].astype('Int64')
df_note['linenb']= df_note['linenb'].astype('Int64')
df_note.head(4)

,id,linenb,cod_causa,testo_originale,intervallo_causa,codeonly,linecoded
0,1,0,R579,COLLASSO CARDIOCIRCOLATORIO,NaN,NaN,1
1,1,2,C799(2Months),METASTASI,02M,NaN,1
2,1,3,C159(4Months),ADENOCARCINOMA ESOFAGEO,04M,NaN,1
3,2,0,R570,COLLASSO CARDIORESPIRATORIO ACUTO,NaN,NaN,1


In [4]:
# REMOVE any parenthesis and their contents, trim whitespace, and insert as a new column right after 'cod_causa'
cleaned = (
    df_note['cod_causa']
    .str.replace(r'\s*\([^)]*\)', '', regex=True)  # strip “ (… )” and any preceding spaces
    .str.strip()                                    # remove leading/trailing spaces
)

insert_loc = df_note.columns.get_loc('cod_causa') + 1
df_note.insert(insert_loc, 'cod_causa_clean', cleaned)

df_note.head(7)

,id,linenb,cod_causa,cod_causa_clean,testo_originale,intervallo_causa,codeonly,linecoded
0,1,0,R579,R579,COLLASSO CARDIOCIRCOLATORIO,NaN,NaN,1
1,1,2,C799(2Months),C799,METASTASI,02M,NaN,1
2,1,3,C159(4Months),C159,ADENOCARCINOMA ESOFAGEO,04M,NaN,1
3,2,0,R570,R570,COLLASSO CARDIORESPIRATORIO ACUTO,NaN,NaN,1
4,2,1,I679,I679,CEREBROPATIA VASCOLARE,NaN,NaN,1
5,2,2,G409,G409,EPILESSIA,NaN,NaN,1
6,2,3,J961; J459,J961; J459,INSUFFICIENZA RESPIRATORIA CRONICA; ASMA,NaN,NaN,1


In [5]:
# prendiamo solo righe con UN codice , NON codici multipli ( legati a piu malattie del paziente)

df_single = df_note[
    df_note['cod_causa_clean'].str.match(r'^[A-Za-z0-9]+$', na=False)
]
df_single.head(5)

,id,linenb,cod_causa,cod_causa_clean,testo_originale,intervallo_causa,codeonly,linecoded
0,1,0,R579,R579,COLLASSO CARDIOCIRCOLATORIO,NaN,NaN,1
1,1,2,C799(2Months),C799,METASTASI,02M,NaN,1
2,1,3,C159(4Months),C159,ADENOCARCINOMA ESOFAGEO,04M,NaN,1
3,2,0,R570,R570,COLLASSO CARDIORESPIRATORIO ACUTO,NaN,NaN,1
4,2,1,I679,I679,CEREBROPATIA VASCOLARE,NaN,NaN,1


In [6]:
# alcune righe hanno codici non riconoscibili, piu di 4 caratteri, esempio:
df_single[ df_single['cod_causa_clean'].str.len() >4 ].head(3)

,id,linenb,cod_causa,cod_causa_clean,testo_originale,intervallo_causa,codeonly,linecoded
18,7,0,I2199(1Days),I2199,INFARTO MIOCARDICO ACUTO,01G,NaN,1
38,14,2,I6409,I6409,ICTUS CEREBRALE,NaN,NaN,1
74,25,0,I6409(1Days),I6409,ICTUS CEREBRI,01G,NaN,1


In [7]:
# TOGLIAMO RIGHE con 'cod_causa_clean' con piu di 3 caratteri, che non nesistono per noi 

df_single= df_single[ df_single['cod_causa_clean'].str.len() <= 4 ]

In [8]:
# Ora ci sono sia codici da 4 ( sotto categorie ) che da 3 carateri ( categorie )
#
df_single[ df_single['cod_causa_clean'].str.len() <= 3 ].head(3)


,id,linenb,cod_causa,cod_causa_clean,testo_originale,intervallo_causa,codeonly,linecoded
12,5,3,R54(7Days),R54,CACHESSIA SENILE,07G,NaN,1
20,7,3,I10(13Years),I10,IPERTENSIONE ARTERIOSA,13A,NaN,1
24,8,3,I10(20Years),I10,IPERTENSIONE ARTERIOSA,20A,NaN,1


In [9]:
#  Formiamo coonna "categoria_clean"  (la categoria ha 3 caratteri ) :
#  togliendo ultimo numero di 'cod_causa_clean' se sono 4 caratteri,
#  altrimenti lo lascia cosi perche 3 caratteri sono gia categoria 

# calcola la posizione subito dopo 'cod_causa_clean'
pos = df_single.columns.get_loc('cod_causa_clean') + 1

# crea la nuova serie: se la lunghezza è 4, togli l’ultimo carattere, altrimenti lasciala invariata
categoria = df_single['cod_causa_clean'].astype(str).apply(lambda x: x[:-1] if len(x) >= 4 else x)

# inserisci la colonna nella posizione desiderata
df_single.insert(pos, 'categoria_clean', categoria)


In [10]:
print(df_single.shape)
df_single

(186555, 9)


,id,linenb,cod_causa,cod_causa_clean,categoria_clean,testo_originale,intervallo_causa,codeonly,linecoded
0,1,0,R579,R579,R57,COLLASSO CARDIOCIRCOLATORIO,NaN,NaN,1
1,1,2,C799(2Months),C799,C79,METASTASI,02M,NaN,1
2,1,3,C159(4Months),C159,C15,ADENOCARCINOMA ESOFAGEO,04M,NaN,1
3,2,0,R570,R570,R57,COLLASSO CARDIORESPIRATORIO ACUTO,NaN,NaN,1
4,2,1,I679,I679,I67,CEREBROPATIA VASCOLARE,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
253619,72345,0,I469,I469,I46,ARRESTO CARDIORESPIRATORIO,NaN,NaN,1
253620,72345,2,J960(3Days),J960,J96,INSUFFICIENZA RESPIRATORIA ACUTA,03G,NaN,1
253621,72345,3,J841(4Years),J841,J84,FIBROSI POLMONARE,04A,NaN,1
253623,72346,2,C786(4Months),C786,C78,CARCINOSI PERITONEALI ASCITOGENA,04M,NaN,1


In [11]:
df_single.index[5]

np.int64(5)

### Test-Data Results

In [15]:
# per ora 10 per velocità , poi torniamo su 1000 eccetera

df_noteSample10 = df_single.sample(n=30, random_state=10)
df_noteSample10.head(5)

,id,linenb,cod_causa,cod_causa_clean,categoria_clean,testo_originale,intervallo_causa,codeonly,linecoded
23577,6657,3,C900(1Months),C900,C90,MIELOMA MULTIPLO,01M,NaN,1
170547,49409,1,R402(10Days),R402,R40,COMA,10G,NaN,1
236540,67544,2,R092,R092,R09,INSUFFICIENZA CARDIO RESPIRATORIA,NaN,NaN,1
94008,27525,3,R960,R960,R96,MORTE IMPROVVISA,NaN,NaN,1
250559,71471,2,N185,N185,N18,INSUFFICIENZA RENALE CRONICA END STAGE,NaN,NaN,1


In [14]:
# Multiple Query seeking 
from semantic_search.qdrant import SemanticSeeker

sk = SemanticSeeker(
    collection_name="istat-icd10_Factory_0",
    model_id="text-embedding-ada-002",
    model_type="openai",
    vector_size=1536,       # ada-002 will default to its fixed 1536
)

queries_list  = ["Scarlattina", "Epilessia", "Polmonite"]
search_outputs= sk.search_many(queries= queries_list, top_k= 2)
search_outputs

Creating embeddings...: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it]


[SearchOutput(results=[RetrievedPoint(id=32, metadata={'label_id': 'A38', 'label_text': 'Scarlattina', 'father_label_id': 4, 'g_father_label_id': 1, 'source_text': 'Scarlattina: Scarlattina'}, score=0.97108066), RetrievedPoint(id=159, metadata={'label_id': 'B86', 'label_text': 'Scabbia', 'father_label_id': 18, 'g_father_label_id': 1, 'source_text': 'Scabbia: Scabbia'}, score=0.84946114)]),
 SearchOutput(results=[RetrievedPoint(id=519, metadata={'label_id': 'G40', 'label_text': 'Epilessia', 'father_label_id': 6, 'g_father_label_id': 6, 'source_text': 'Epilessia: Epilessia e sindromi epilettiche idiopatiche, definite per localizzazione (focale) (parziale), con crisi ad esordio focale Epilessia e sindromi epilettiche sintomatiche, definite per localizzazione (focale(i)) (parziale(i)), con crisi parziali semplici Epilessia e sindromi epilettiche sintomatiche, definite per localizzazione (focale(i)) (parziale(i)), con crisi parziali complesse Epilessia e sindromi epilettiche generalizzate i

In [132]:
search_outputs[0]

SearchOutput(results=[RetrievedPoint(id=32, metadata={'label_id': 'A38', 'label_text': 'Scarlattina', 'father_label_id': 4, 'g_father_label_id': 1, 'source_text': 'Scarlattina: Scarlattina'}, score=0.9711024), RetrievedPoint(id=159, metadata={'label_id': 'B86', 'label_text': 'Scabbia', 'father_label_id': 18, 'g_father_label_id': 1, 'source_text': 'Scabbia: Scabbia'}, score=0.8494627)])

In [130]:
search_outputs[0].results

[RetrievedPoint(id=32, metadata={'label_id': 'A38', 'label_text': 'Scarlattina', 'father_label_id': 4, 'g_father_label_id': 1, 'source_text': 'Scarlattina: Scarlattina'}, score=0.9711024),
 RetrievedPoint(id=159, metadata={'label_id': 'B86', 'label_text': 'Scabbia', 'father_label_id': 18, 'g_father_label_id': 1, 'source_text': 'Scabbia: Scabbia'}, score=0.8494627)]

In [133]:
search_outputs[0].results[1]

RetrievedPoint(id=159, metadata={'label_id': 'B86', 'label_text': 'Scabbia', 'father_label_id': 18, 'g_father_label_id': 1, 'source_text': 'Scabbia: Scabbia'}, score=0.8494627)

In [ ]:
# da  lavoro processo dure 47 secondi
# da casa ( wifi fastweb ) 35 sec

In [ ]:
def retrieve_topk_df(
    seeker: SemanticSeeker,
    df: pd.DataFrame,
    batch_size=64,
    text_col="testo_originale",
    k=3
):
    queries = df[text_col].tolist()
    search_outputs = seeker.search_many(queries, top_k=k, batch_size=batch_size)

    data = []
    for i, out in enumerate(search_outputs):
        row = {
            "id_query": df.index[i],                       # o df.iloc[i]["id"]
            "label_id_true": df.iloc[i]["categoria_clean"],
            "testo_query": queries[i],
            "----": "----"
        }

        # aggiunge pred_x e score_x
        for rank in range(k):
            if rank < len(out.results):
                point = out.results[rank]
                row[f"pred_{rank+1}"]  = point.metadata["label_id"]
                row[f"score_{rank+1}"] = point.score
            else:
                row[f"pred_{rank+1}"]  = None
                row[f"score_{rank+1}"] = None

        data.append(row)

    return pd.DataFrame(data)


In [35]:
out_df= retrieve_topk_df(seeker=sk, batch_size=100 , df= df_noteSample10,  text_col="testo_originale", k=3)


Creating embeddings...: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


In [39]:
pred_df = out_df.copy()
pred_df.head()

,id_query,label_id_true,testo_query,----,pred_1,score_1,pred_2,score_2,pred_3,score_3
0,23577,C90,MIELOMA MULTIPLO,----,G35,0.867499,C90,0.861062,B91,0.835032
1,170547,R40,COMA,----,E15,0.829630,R40,0.816859,B19,0.815767
2,236540,R09,INSUFFICIENZA CARDIO RESPIRATORIA,----,J96,0.869884,I50,0.868136,I11,0.858694
3,94008,R96,MORTE IMPROVVISA,----,R98,0.872577,R96,0.861974,R95,0.855375
4,250559,N18,INSUFFICIENZA RENALE CRONICA END STAGE,----,N18,0.912072,N19,0.872557,N17,0.864105


#### Accuracy

In [49]:
k=3

for n in range(1, k + 1):
    pred_cols = [f"pred_{r}" for r in range(1, n + 1)]
    print(pred_cols)
    

['pred_1']
['pred_1', 'pred_2']
['pred_1', 'pred_2', 'pred_3']


In [48]:
for n in range(1, k + 1):
    pred_cols = [f"pred_{r}" for r in range(1, n + 1)]
    pred_df[f"hit@{n}"] = pred_df.apply(lambda row: row['label_id_true'] in row[pred_cols].values, axis=1) 

pred_df.tail(4)

,id_query,label_id_true,testo_query,----,pred_1,score_1,pred_2,score_2,pred_3,score_3,hit@1,hit@2,hit@3
26,843,R54,CACHESSIA SENILE,----,R64,0.860635,R54,0.860150,H25,0.855457,False,True,True
27,135822,I48,FIBRILLAZIONE ATRIALE PAROSSISTICA,----,I48,0.877321,I47,0.842575,I49,0.838333,True,True,True
28,153825,I46,ARRESTO CARDIOCIRCOLATORIO ACUTO,----,I46,0.881573,I24,0.863644,I21,0.853019,True,True,True
29,248080,C78,METASTASI EPATICHE,----,C22,0.871403,C23,0.865563,C24,0.859788,False,False,False


In [51]:
def evaluate_hits(df_pred: pd.DataFrame, k: int) -> pd.DataFrame:
    """
    Adds columns hit@1 … hit@k to df_pred and returns the same DF.
    Each hit@n is True if the true label appears in the first n predictions.
    """
    for n in range(1, k + 1):
        pred_cols = [f"pred_{r}" for r in range(1, n + 1)]
        df_pred[f"hit@{n}"] = df_pred.apply(
            lambda row: row["label_id_true"] in row[pred_cols].values,
            axis=1,
        )
    return df_pred


In [70]:
evaluate_hits(pred_df, 3)

,id_query,label_id_true,testo_query,----,pred_1,score_1,pred_2,score_2,pred_3,score_3,hit@1,hit@2,hit@3
0,23577,C90,MIELOMA MULTIPLO,----,G35,0.867499,C90,0.861062,B91,0.835032,False,True,True
1,170547,R40,COMA,----,E15,0.829630,R40,0.816859,B19,0.815767,False,True,True
2,236540,R09,INSUFFICIENZA CARDIO RESPIRATORIA,----,J96,0.869884,I50,0.868136,I11,0.858694,False,False,False
3,94008,R96,MORTE IMPROVVISA,----,R98,0.872577,R96,0.861974,R95,0.855375,False,True,True
4,250559,N18,INSUFFICIENZA RENALE CRONICA END STAGE,----,N18,0.912072,N19,0.872557,N17,0.864105,True,True,True
5,209252,D48,MALATTIA NEOPLASTICA,----,C90,0.841206,C07,0.839061,C97,0.838772,False,False,False
6,132656,J18,POLMONITE,----,J13,0.836219,B91,0.835726,J14,0.834360,False,False,False
7,212927,C18,NEOPLASIA COLON,----,D12,0.818980,P77,0.818761,N40,0.818226,False,False,False
8,63472,I46,ARRESTO CARDIOCIRCOLATORIO,----,I46,0.872571,I22,0.842252,I47,0.839590,True,True,True
9,26002,I48,FIBRILLAZIONE ATRIALE 20G,----,I48,0.851510,I49,0.811636,B92,0.806937,True,True,True


In [ ]:
pred_df[ "hit@1" ].sum()



np.int64(12)

In [71]:
pred_df[ "hit@2" ].sum()

np.int64(19)

In [58]:
hit_col= [f"hit@{n}" for n in range(1, k+1)] 
hit_col

['hit@1', 'hit@2', 'hit@3']

In [61]:
metrics = pred_df[[f"hit@{n}" for n in range(1, 4)]].mean()
print(metrics)

hit@1    0.400000
hit@2    0.633333
hit@3    0.633333
dtype: float64


In [ ]:
# Parallelizzare richieste

# qui 15 secondi

# 0) qui hai già:
#    - df_noteSample10               (DataFrame con 1 000 righe)
#    - seeker                        (SemanticSeeker pronto)
#    - retrieve_topk(...)            (se vuoi poi costruire il DF)

# 1) lista delle query
queries = df_noteSample10["testo_originale"].astype(str).tolist()

# 2) parametri per la prova
n_threads   = 8           # = numero di core fisici o logici che vuoi usare
top_k       = 3
inner_batch = 1000        # batch size passato a search_many

# 3) funzione wrapper (fuori da SemanticSeeker)
from concurrent.futures import ThreadPoolExecutor
import numpy as np, time

def run_chunk(chunk):
    return sk.search_many(list(chunk), top_k=top_k, batch_size=inner_batch)

# 4) suddividi le query in n_threads porzioni (array_split mantiene l’ordine)
chunks = np.array_split(queries, n_threads)

t0 = time.perf_counter()

with ThreadPoolExecutor(max_workers=n_threads) as pool:
    futures  = [pool.submit(run_chunk, c) for c in chunks]
    outputs  = [out for f in futures for out in f.result()]  # concateni i risultati

t1 = time.perf_counter()
print(f"Tempo totale: {t1 - t0:,.2f} s")


In [58]:
outputs

[SearchOutput(results=[RetrievedPoint(id=516, metadata={'label_id': 'G35', 'label_text': 'Sclerosi multipla', 'father_label_id': 5, 'g_father_label_id': 6, 'source_text': 'Sclerosi multipla: Sclerosi multipla'}, score=0.8674552), RetrievedPoint(id=251, metadata={'label_id': 'C90', 'label_text': 'Mieloma multiplo e tumori plasmacellulari maligni', 'father_label_id': 14, 'g_father_label_id': 2, 'source_text': 'Mieloma multiplo e tumori plasmacellulari maligni: Mieloma multiplo Leucemia plasmacellulare Plasmocitoma, extramidollare'}, score=0.8610153), RetrievedPoint(id=164, metadata={'label_id': 'B91', 'label_text': 'Sequele di poliomielite', 'father_label_id': 19, 'g_father_label_id': 1, 'source_text': 'Sequele di poliomielite: Sequele di poliomielite'}, score=0.8349946)]),
 SearchOutput(results=[RetrievedPoint(id=354, metadata={'label_id': 'E15', 'label_text': 'Coma ipoglicemico non diabetico', 'father_label_id': 3, 'g_father_label_id': 4, 'source_text': 'Coma ipoglicemico non diabetico